In [1]:
!pip install pyspark
!pip install -q kaggle # los datos para el laboratorio se alojan alli

## ⭐ Importar librerias

In [2]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import zipfile
import os
import pandas as pd

### se sube aquí el JSON

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"cgonzalezm","key":"36bd5b66ff8c309d5e8bb74416fdaeac"}'}

### se guarda el archivo JSON en una carpet oculta

In [4]:
# Crear la carpeta oculta .kaggle en tu home, allí se guardará el archivo JSON
!mkdir -p ~/.kaggle

# Mover el archivo kaggle.json a esa carpeta
!mv kaggle.json ~/.kaggle/

# cambia los permisos de acceso, solo al usuario actual
!chmod 600 ~/.kaggle/kaggle.json


### con acceso a Kaggle, ya se descarga los datos de arevel/chess-games

In [5]:
! kaggle datasets download arevel/chess-games --force

Dataset URL: https://www.kaggle.com/datasets/arevel/chess-games
License(s): CC0-1.0
 99% 1.43G/1.45G [00:14<00:00, 99.1MB/s]
100% 1.45G/1.45G [00:14<00:00, 105MB/s] 


### Se descargo un zip, es hora de descomprimir el archivo

In [6]:
for file in os.listdir():
  if file.endswith('.zip'):
    zip_ref =zipfile.ZipFile(file, 'r')
    zip_ref.extractall()
    zip_ref.close()

In [7]:
"""spark= SparkSession.builder\
      .master("local")\
      .appName("Colab")\
      .config('spark.ui.port', '4050')\
      .getOrCreate()"""

'spark= SparkSession.builder      .master("local")      .appName("Colab")      .config(\'spark.ui.port\', \'4050\')      .getOrCreate()'

In [8]:
spark = (SparkSession.builder
    .master("local[*]")
    .appName("SolucionTaller")
    .getOrCreate()
)

In [9]:
# Definir el esquema correcto para cada uno de los tipos de datos
# Para los campos que tienen cadena de texto dejarlos todos en mayuscula
# Crear una nueva columna que indique el año en el que tuvo lugar el evento
# Cantidad de enventos por año

In [10]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import *

In [11]:
schema = StructType([
    StructField("Event", StringType(), True),
    StructField("White", StringType(), True),
    StructField("Black", StringType(), True),
    StructField("Result", StringType(), True),
    StructField("UTCDate", StringType(), True),
    StructField("UTCTime", StringType(), True),
    StructField("WhiteElo", IntegerType(), True),
    StructField("BlackElo", IntegerType(), True),
    StructField("WhiteRatingDiff", IntegerType(), True),
    StructField("BlackRatingDiff", IntegerType(), True),
    StructField("ECO", StringType(), True),
    StructField("Opening", StringType(), True),
    StructField("TimeControl", StringType(), True),
    StructField("Termination", StringType(), True),
    StructField("AN" ,StringType(), True)]
)



In [27]:
df = spark.read.csv('/content/chess_games.csv', header =True, schema=schema)

In [28]:
string_cols = [f.name for f in df.schema.fields if isinstance(f.dataType, StringType)]

df = df.select(
    *[
        F.upper(F.col(c)).alias(c) if c in string_cols else F.col(c)
        for c in df.columns
    ]
)

In [29]:
df  = df.select("*", F.substring(F.col("UTCDate"),1,4).alias("Year"))

In [15]:
df.groupBy("Year").count().show()

+----+-------+
|Year|  count|
+----+-------+
|2016|6256184|
+----+-------+



In [30]:
# Crear una UDF que eliminr los espacios que encuentran en la izquierda y en la derecha del string

def trim_spaces(s):
  return s.strip()

trim_spaces_udf = F.udf(trim_spaces, StringType())

string_cols = [f.name for f in df.schema.fields if isinstance(f.dataType, StringType)]

df = df.select(
    *[
        trim_spaces_udf(F.col(c)).alias(c) if c in string_cols else F.col(c)
        for c in df.columns
    ]
)




In [24]:
df.show()

+----------------+---------------+---------------+------+----------+--------+--------+--------+---------------+---------------+---+--------------------+-----------+------------+--------------------+----+
|           Event|          White|          Black|Result|   UTCDate| UTCTime|WhiteElo|BlackElo|WhiteRatingDiff|BlackRatingDiff|ECO|             Opening|TimeControl| Termination|                  AN|Year|
+----------------+---------------+---------------+------+----------+--------+--------+--------+---------------+---------------+---+--------------------+-----------+------------+--------------------+----+
|       CLASSICAL|        EISAAAA|       HAMID449|   1-0|2016.06.30|22:00:01|    1901|    1896|           NULL|           NULL|D10|        SLAV DEFENSE|      300+5|TIME FORFEIT|1. D4 D5 2. C4 C6...|2016|
|           BLITZ|         GO4JAS|     SERGEI1973|   0-1|2016.06.30|22:00:01|    1641|    1627|           NULL|           NULL|C20|KING'S PAWN OPENI...|      300+0|      NORMAL|1. E4 E

In [31]:
# Tiempo promedio por cada uno de los eventos
df = df.where(df.TimeControl != '-')
df = df.withColumn(
    "TimeControl",
    F.expr("TRY_CAST(substring(TimeControl, 1, length(TimeControl) - 2) AS INT)")
)
df.createOrReplaceTempView("Events")

avg_time = spark.sql("""SELECT event,
  AVG(TimeControl) AS AVG_TimeControl
        FROM Events
        GROUP BY event
""")



# Dividir en el dataframe principal cada uno los registros por el tiempo promedion (Join usando el evento llave)
df = df.join(avg_time, on="Event", how="left")

spark.catalog.dropTempView("Events")

df.select("Event",(F.col("TimeControl")/F.col("AVG_TimeControl")).alias("Result")).show()

+----------------+-------------------+
|           Event|             Result|
+----------------+-------------------+
|       CLASSICAL|0.43438503812781915|
|       CLASSICAL| 1.3031551143834574|
|       CLASSICAL| 0.6081390533789468|
|BLITZ TOURNAMENT| 0.7804603152830822|
|BLITZ TOURNAMENT| 0.7804603152830822|
|BLITZ TOURNAMENT| 0.7804603152830822|
|BLITZ TOURNAMENT| 0.7804603152830822|
|BLITZ TOURNAMENT| 0.7804603152830822|
|BLITZ TOURNAMENT| 1.3007671921384703|
|BLITZ TOURNAMENT| 0.7804603152830822|
|BLITZ TOURNAMENT| 1.3007671921384703|
|BLITZ TOURNAMENT| 0.7804603152830822|
|BLITZ TOURNAMENT| 0.7804603152830822|
|BLITZ TOURNAMENT| 0.7804603152830822|
|BLITZ TOURNAMENT| 0.7804603152830822|
|BLITZ TOURNAMENT| 0.7804603152830822|
|BLITZ TOURNAMENT| 1.3007671921384703|
|BLITZ TOURNAMENT| 1.3007671921384703|
|BLITZ TOURNAMENT| 1.3007671921384703|
|BLITZ TOURNAMENT| 1.3007671921384703|
+----------------+-------------------+
only showing top 20 rows


In [18]:
df.createOrReplaceTempView("lab")